GSMMs part 1

Constructing the COBRApy environment

In [56]:
import cobra
from cobra.io import write_sbml_model, save_json_model

# 1. Creating a new model
model = cobra.Model("Assignment752_Model_Complete")

# 2. Definition of Metabolites
O = cobra.Metabolite('O', compartment='c')
C = cobra.Metabolite('C', compartment='c')
N = cobra.Metabolite('N', compartment='c')
B = cobra.Metabolite('B', compartment='c')
P = cobra.Metabolite('P', compartment='c')

O_e = cobra.Metabolite('O_e', compartment='e')
C_e = cobra.Metabolite('C_e', compartment='e')
N_e = cobra.Metabolite('N_e', compartment='e')
B_e = cobra.Metabolite('B_e', compartment='e')
P_e = cobra.Metabolite('P_e', compartment='e')

# 3. specification of reactions with GPR Rules and Stoichiometry 
#Demand reactions
DM_C = cobra.Reaction('DM_C_e')
DM_C.lower_bound = -1000
DM_C.upper_bound = 0
DM_C.add_metabolites({C_e: -1})

DM_O = cobra.Reaction('DM_O_e')
DM_O.lower_bound = -1000
DM_O.upper_bound = 0
DM_O.add_metabolites({O_e: -1})

DM_N = cobra.Reaction('DM_N_e')
DM_N.lower_bound = -1000
DM_N.upper_bound = 0
DM_N.add_metabolites({N_e: -1})

# Main reactions 
v1 = cobra.Reaction('v1')
v1.add_metabolites({O_e: -1, O: 1})
v1.lower_bound = 0
v1.upper_bound = 1000
v1.gene_reaction_rule = "gene1"

v2 = cobra.Reaction('v2')
v2.add_metabolites({C_e: -1, C: 1})
v2.lower_bound = 0
v2.upper_bound = 1000
v2.gene_reaction_rule = "gene1 or gene2 or gene3"

v3 = cobra.Reaction('v3')
v3.add_metabolites({N_e: -1, N: 1})
v3.lower_bound = 0
v3.upper_bound = 1000
v3.gene_reaction_rule = "gene2 and gene4"

v4 = cobra.Reaction('v4')
v4.add_metabolites({O: -1, C: -1, N: -1, B: 1})
v4.lower_bound = 0
v4.upper_bound = 1000
v4.gene_reaction_rule = "(gene9 and gene10) or gene8"

v5 = cobra.Reaction('v5')
v5.add_metabolites({C: -1, N: -2, B: 1, P: 2})
v5.lower_bound = -1000  # Reversible
v5.upper_bound = 1000
v5.gene_reaction_rule = "gene7 and (gene5 or gene6)"

v6 = cobra.Reaction('v6')
v6.add_metabolites({B: -1, B_e: 1})
v6.lower_bound = 0
v6.upper_bound = 1000
v6.gene_reaction_rule = "gene14"

v7 = cobra.Reaction('v7')
v7.add_metabolites({P: -1, P_e: 1})
v7.lower_bound = 0
v7.upper_bound = 1000
v7.gene_reaction_rule = "gene11 and gene12 and gene13"

# Sink reactions (exportion of metabolites)
SK_B = cobra.Reaction('SK_B_e')
SK_B.add_metabolites({B_e: -1})
SK_B.lower_bound = 0
SK_B.upper_bound = 1000

SK_P = cobra.Reaction('SK_P_e')
SK_P.add_metabolites({P_e: -1})
SK_P.lower_bound = 0
SK_P.upper_bound = 1000

# 4. Adding the reactions to the model
model.add_reactions([DM_C, DM_O, DM_N, v1, v2, v3, v4, v5, v6, v7, SK_B, SK_P])

# Saving of model 
from cobra.io import write_sbml_model, save_json_model

# specified path where model is saved to
save_path = r"/Users/alexbeggs/Dropbox/Bioinformatics Msc/LIFE 752 - Computational/life752_cycle2/GSMs_assessment/"

# xml file save
write_sbml_model(model, f"{save_path}assignment_model_complete_14.xml")

# JSON file save
save_json_model(model, f"{save_path}assignment_model_complete_14.json")


Q3. FBA analysis

In [57]:
import cobra
from cobra.io import write_sbml_model, save_json_model

# path to model specified
model_path = r"/Users/alexbeggs/Dropbox/Bioinformatics Msc/LIFE 752 - Computational/life752_cycle2/GSMs_assessment/assignment_model_complete_14.xml"

# loading of the model
model = cobra.io.read_sbml_model(model_path)

# 1. v6 is set as the objective function (export of B)
model.objective = "v6"  # Set v6 as the objective function

# 2. Maximum Uptake Constraints are set for N, C, and O
# Uptake of N (DM_N_e) - maximum of 10 mmol/gDW/h
model.reactions.get_by_id("DM_N_e").lower_bound = -10
model.reactions.get_by_id("DM_N_e").upper_bound = 0

# Uptake of C (DM_C_e) - maximum of 15 mmol/gDW/h
model.reactions.get_by_id("DM_C_e").lower_bound = -15
model.reactions.get_by_id("DM_C_e").upper_bound = 0

# Uptake of O (DM_O_e) - maximum of 10 mmol/gDW/h
model.reactions.get_by_id("DM_O_e").lower_bound = -10
model.reactions.get_by_id("DM_O_e").upper_bound = 0

# updated model saved to file path
save_path = r"/Users/alexbeggs/Dropbox/Bioinformatics Msc/LIFE 752 - Computational/life752_cycle2/GSMs_assessment"
write_sbml_model(model, f"{save_path}/assignment_model_updated_14.xml")  
save_json_model(model, f"{save_path}/assignment_model_updated_14.json")  




No objective coefficients in model. Unclear what should be optimized


Printing of FBA analysis results

In [58]:
import cobra
from cobra.io import read_sbml_model

# 1. specified path to FBA model
model_path = r"/Users/alexbeggs/Dropbox/Bioinformatics Msc/LIFE 752 - Computational/life752_cycle2/GSMs_assessment/assignment_model_updated_14.xml"
model = read_sbml_model(model_path)

# 2. FBA analysis ran
solution = model.optimize()

# 3. printing of FBA analysis results
print("Flux Distribution from FBA:\n")
print(solution.fluxes)


Flux Distribution from FBA:

DM_C_e   -10.0
DM_O_e   -10.0
DM_N_e   -10.0
v1        10.0
v2        10.0
v3        10.0
v4        10.0
v5         0.0
v6        10.0
v7         0.0
SK_B_e    10.0
SK_P_e     0.0
Name: fluxes, dtype: float64


In [59]:
import escher
from escher import Builder

# path defined for map 
map_path = r"/Users/alexbeggs/Dropbox/Bioinformatics Msc/LIFE 752 - Computational/life752_cycle2/GSMs_assessment/assignment_map.json"

# escher map loaded with model applied
builder = Builder(
    model=model,             
    map_json=map_path,        
    reaction_data=solution.fluxes  #intergrated FBA results
)

# Displaying of the map
builder



Builder(reaction_data={'DM_C_e': -10.0, 'DM_O_e': -10.0, 'DM_N_e': -10.0, 'v1': 10.0, 'v2': 10.0, 'v3': 10.0, …

Q4. Exploration of gene deletion impact on the model

In [60]:
import cobra
from cobra.io import read_sbml_model

# path to model defined
model_path = r"/Users/alexbeggs/Dropbox/Bioinformatics Msc/LIFE 752 - Computational/life752_cycle2/GSMs_assessment/assignment_model_updated_14.xml"

# Function reloads model everytime to ensure consistency
def reload_model():
    model = read_sbml_model(model_path)
    return model


In [61]:
import cobra
import pandas as pd
from cobra.io import save_json_model, load_json_model
from cobra.manipulation import knock_out_model_genes

# saves the working model as json file
save_json_model(model, "assignment_model_updated_14.json")  


#Distributions for each knockout and wild-type stored
all_fluxes = {}

# no gene deletion model saved
model = load_json_model("assignment_model_updated_14.json")

wild_type_solution = model.optimize()
wild_type_fluxes = wild_type_solution.fluxes.to_frame()
wild_type_fluxes.columns = ["wild_type_flux"]
all_fluxes["wild_type"] = wild_type_fluxes
print(f"Wild-type Objective (v6): {wild_type_solution.objective_value}")

# Function created to simulate gene deletions from FBA model
def simulate_deletion(gene_id):
    model = load_json_model("assignment_model_updated_14.json")

    with model:
        knock_out_model_genes(model, [gene_id])
        solution = model.optimize()
        fluxes = solution.fluxes.to_frame()
        fluxes.columns = [f"{gene_id}_flux"]
        all_fluxes[gene_id] = fluxes
        print(f"Gene: {gene_id}, Objective (v6): {solution.objective_value}")
        return fluxes

# gene deletion simulations ran
simulate_deletion("gene1")
simulate_deletion("gene5")
simulate_deletion("gene14")

# merging of ouputs into table
summary_flux_table = pd.concat(all_fluxes.values(), axis=1)

# printing of table
print("\nFlux Comparison Table:")
print(summary_flux_table)


Wild-type Objective (v6): 10.0
Gene: gene1, Objective (v6): 5.0
Gene: gene5, Objective (v6): 10.0
Gene: gene14, Objective (v6): 0.0

Flux Comparison Table:
        wild_type_flux  gene1_flux  gene5_flux  gene14_flux
DM_C_e           -10.0        -5.0       -10.0          0.0
DM_O_e           -10.0         0.0       -10.0          0.0
DM_N_e           -10.0       -10.0       -10.0          0.0
v1                10.0         0.0        10.0          0.0
v2                10.0         5.0        10.0          0.0
v3                10.0        10.0        10.0          0.0
v4                10.0         0.0        10.0          0.0
v5                 0.0         5.0         0.0          0.0
v6                10.0         5.0        10.0          0.0
v7                 0.0        10.0         0.0          0.0
SK_B_e            10.0         5.0        10.0          0.0
SK_P_e             0.0        10.0         0.0          0.0


In [62]:

import pandas as pd

# Baseline FBA Results (Before Deletion any gene deletions
baseline = {
    "Reaction": ["DM_C_e", "DM_O_e", "DM_N_e", "v1", "v2", "v3", "v4", "v5", "v6", "v7", "SK_B_e", "SK_P_e"],
    "Baseline Flux": [ -10, -10, -10, 10, 10, 10, 10, 0, 10, 0, 10, 0]
}

# Results following Each Gene Deletion
results = {
    "Gene1 Deleted": [0, -5, -10, 0, 5, 10, 0, 5, 5, 10, 5, 10],
    "Gene5 Deleted": [-10, -10, -10, 10, 10, 10, 10, 0, 10, 0, 10, 0],
    "Gene14 Deleted": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
}

# Creation of a DataFrame for a clear table
df = pd.DataFrame(baseline)
df["Gene1 Deleted"] = results["Gene1 Deleted"]
df["Gene5 Deleted"] = results["Gene5 Deleted"]
df["Gene14 Deleted"] = results["Gene14 Deleted"]

# table displayed
df.style.set_caption("Comparison of FBA Results Before and After Gene Deletion simulations")


,Reaction,Baseline Flux,Gene1 Deleted,Gene5 Deleted,Gene14 Deleted
0,DM_C_e,-10,0,-10,0
1,DM_O_e,-10,-5,-10,0
2,DM_N_e,-10,-10,-10,0
3,v1,10,0,10,0
4,v2,10,5,10,0
5,v3,10,10,10,0
6,v4,10,0,10,0
7,v5,0,5,0,0
8,v6,10,5,10,0
9,v7,0,10,0,0


Q5. Impact of gene deletions (1-14) on the production of P_e with v6 as the objective function.

In [63]:
import cobra
import pandas as pd
from cobra.io import load_json_model
from cobra.manipulation import knock_out_model_genes

# table formatting
try:
    from tabulate import tabulate
    use_tabulate = True
except ImportError:
    use_tabulate = False

# Loading of the model
model = load_json_model("assignment_model_updated_14.json")

# Seting of v6 as the objective (export of B)
model.objective = "v6"

# Dictionary to store P_e flux only
gene_deletion_results = {}

# Wild-type (no deletion) 
solution_wt = model.optimize()
wt_flux_pe = solution_wt.fluxes.get("SK_P_e", 0.0)
gene_deletion_results["wild_type"] = wt_flux_pe
print(f"Wild-type: SK_P_e flux = {wt_flux_pe}")

# Gene knockouts (gene1 to gene14)
genes_to_test = [f"gene{i}" for i in range(1, 15)]

for gene_id in genes_to_test:
    with model.copy() as temp_model:
        knock_out_model_genes(temp_model, [gene_id])
        solution = temp_model.optimize()
        flux_pe = solution.fluxes.get("SK_P_e", 0.0)
        gene_deletion_results[gene_id] = flux_pe
        print(f"Deleted {gene_id}: SK_P_e flux = {flux_pe}")

# Conversion to dataFrame
df_results = pd.DataFrame.from_dict(
    gene_deletion_results,
    orient='index',
    columns=["P_e Flux (mmol/gDW/hr)"]
)
df_results.index.name = "Gene Deletion"

# Ensuring proper order
df_results = df_results.loc[["wild_type"] + genes_to_test]

# Print table in nice format
print("\nTable 3. Impact of Gene Deletions on P_e Production (Objective: v6):\n")
if use_tabulate:
    print(tabulate(df_results.reset_index(), headers='keys', tablefmt='grid', floatfmt=".2f"))
else:
    print(df_results)




Wild-type: SK_P_e flux = 0.0
Deleted gene1: SK_P_e flux = 10.0
Deleted gene2: SK_P_e flux = 0.0
Deleted gene3: SK_P_e flux = 0.0
Deleted gene4: SK_P_e flux = 0.0
Deleted gene5: SK_P_e flux = 0.0
Deleted gene6: SK_P_e flux = 0.0
Deleted gene7: SK_P_e flux = 0.0
Deleted gene8: SK_P_e flux = 0.0
Deleted gene9: SK_P_e flux = 0.0
Deleted gene10: SK_P_e flux = 0.0
Deleted gene11: SK_P_e flux = 0.0
Deleted gene12: SK_P_e flux = 0.0
Deleted gene13: SK_P_e flux = 0.0
Deleted gene14: SK_P_e flux = 0.0

Table 3. Impact of Gene Deletions on P_e Production (Objective: v6):

               P_e Flux (mmol/gDW/hr)
Gene Deletion                        
wild_type                         0.0
gene1                            10.0
gene2                             0.0
gene3                             0.0
gene4                             0.0
gene5                             0.0
gene6                             0.0
gene7                             0.0
gene8                             0.0
gene9         